In [1]:
from commonfunctions import *
import numpy as np
import cv2
import math
from scipy import ndimage
import math
#import imutils
from skimage.filters import threshold_otsu, threshold_local
from skimage.morphology import binary_erosion, binary_dilation
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
def show(img, factor=1,name="image"):
    """ 
    show an image until the escape key is pressed
    :param factor: scale factor (default 1, half size)
    """
    if factor != 1.0:
        img = cv2.resize(img, (0,0), fx=factor, fy=factor) 

    cv2.imshow(name,img)
    while(1):
        k = cv2.waitKey(0)
        if k==27:    # Esc key to stop
            break
    cv2.destroyAllWindows()

In [7]:
#This notebook aims to find the proper use of feature matching OpenCV function to identify musical notes
chall1 = '/home/ahmad/Desktop/FeatureMatching/Chall1.jpeg'
chall2 = '/home/ahmad/Desktop/FeatureMatching/Chall2.jpg'
box = '/home/ahmad/Desktop/FeatureMatching/PJBox.jpg'
logo = '/home/ahmad/Desktop/FeatureMatching/PJLogo.png'
chall1 = cv2.imread(chall1)
chall2 = cv2.imread(chall2)

box = cv2.imread(box)
logo = cv2.imread(logo)
box = cv2.cvtColor(box, cv2.COLOR_BGR2GRAY)
logo = cv2.cvtColor(logo, cv2.COLOR_BGR2GRAY)
#show(chall1,1,"Chall1")
#show(chall2,1,"Chall2")
#show(box,1,"Box")
#show(logo,1,"Logo")

sift = cv2.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(logo,None)
kp2, des2 = sift.detectAndCompute(box,None)
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=100)   # specifies number of time that trees are recursively traversed, improves precision
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)
# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
#ratio test as per Lowe's paper
good = 0 
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        good +=1
        matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv2.DrawMatchesFlags_DEFAULT)
if (good > 5):
    print("match")
img3 = cv2.drawMatchesKnn(logo,kp1,box,kp2,matches,None,**draw_params)
show(img3, 1, "Compared")

match


In [ ]:
'''
Description of SIFT:
First we compute Difference of Gaussians. Which is we get different gaussian blurs of the image 
and we subtract them from each other.
Then we compute gradients of of the computed neighbourhood of the keypoint => histogram of gradients
This gives us a descriptor vector

'''